# Fake-o-News Detector

This is based on AWS excellent blog [Harness large language models in fake news detection](https://aws.amazon.com/blogs/machine-learning/harness-large-language-models-in-fake-news-detection/).

I have modified to Google's Gemini to enourage more coders out there to look into this area of using the power of technology to do good: by getting rid the bad and the ugly "fake news".  I do encourage all to try AWS's Bedrock and other LLMs to make this area of fake news detection an daily successful applications of Generative AI.

The AWS blog do give this project a good start into [LangChain](https://www.langchain.com/) with LLM.

##References
* https://aws.amazon.com/blogs/machine-learning/harness-large-language-models-in-fake-news-detection/
* https://dev.to/timesurgelabs/how-to-use-googles-gemini-pro-with-langchain-1eje

## Install the necessary packages

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.5/111.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [2]:
!pip install langchainhub

In [3]:
!pip install python-dotenv

In [4]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.6/663.6 kB 11.1 MB/s eta 0:00:00


In [5]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=9fd57f2f652fae36f00cb384dca4a14be9f253438643061a8f85cdf326e6912e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [6]:
!pip -q install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 46.6 MB/s eta 0:00:00


In [7]:
!pip install google-generativeai langchain-google-genai

## Import the needed Libraries

In [8]:
from langchain import hub
from langchain.agents import load_tools

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

from langchain.agents import AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from dotenv import load_dotenv, find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import StuffDocumentsChain
from langchain.chains.llm import LLMChain

from langchain.tools import DuckDuckGoSearchRun

_ = load_dotenv(find_dotenv()) # read local .env file

In [9]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

##Using Google Colab API key storage to securely retieve API key

In [10]:
# Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [11]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Getting the LLM

* Tutorial from [here](https://dev.to/timesurgelabs/how-to-use-googles-gemini-pro-with-langchain-1eje)

In [12]:
def get_llm():
    llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
    return llm

##  Chain-of-Thought prompt chain

In [13]:
def generate_and_print_ChainOfThought(llm, q):
    total_prompt = """"""

    template = """Here is a statement:
    {statement}
    Make a bullet point list of the assumptions you made when given the above statement.\n\n"""
    prompt_template = PromptTemplate(input_variables=["statement"], template=template)
    assumptions_chain = LLMChain(llm=llm, prompt=prompt_template)
    total_prompt = total_prompt + template

    template = """Here is a bullet point list of assertions:
    {assertions}
    For each assertion, determine whether it is true or false. If it is false, explain why.\n\n"""
    prompt_template = PromptTemplate(input_variables=["assertions"], template=template)
    fact_checker_chain = LLMChain(llm=llm, prompt=prompt_template)
    total_prompt = total_prompt + template

    template = """Based on the above assertions, the final response is FALSE if one of the assertions is FALSE. Otherwise, TRUE. You should only respond with TRUE or FALSE.'{}'""".format(q)
    template = """{facts}\n""" + template
    prompt_template = PromptTemplate(input_variables=["facts"], template=template)
    answer_chain = LLMChain(llm=llm, prompt=prompt_template)
    total_prompt = total_prompt + template

    overall_chain = SimpleSequentialChain(chains=[assumptions_chain, fact_checker_chain, answer_chain], verbose=True)
    answer = overall_chain.run(q)

    return answer

##ReAct

Reference on create react agaent could be found [here](https://wikidocs.net/236206)

Tools for LangChain could be found [here](https://python.langchain.com/docs/integrations/tools/)

How to use DuckDuckGo Search could be found [here](https://rito.hashnode.dev/building-langchain-agent-with-tools)

In [14]:
def generate_and_print_ReAct(llm, tl, q):
    print(f'Inside generate_and_print: q = {q}')
    tools = load_tools([tl], llm=llm) # tl = "ddg-search" or "wikipedia"

    # create_react_agent Ref: https://wikidocs.net/236206
    prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    input = """Here is a statement:
    {statement}
    Is this statement is correct? You can use tools to find information if needed.
    The final response is FALSE if the statement is FALSE. Otherwise, TRUE."""

    answer = agent_executor.invoke({"input" : input.format(statement=q)})

    return answer

##Input your Fake News here

Sample Question: "The first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov, who earned her degree from Stanford University in 1968."

Answers could be found in [Harness large language models in fake news detection](https://aws.amazon.com/blogs/machine-learning/harness-large-language-models-in-fake-news-detection/)

In [31]:
fakeNews = input("Enter the news here: ")

Enter the news here: Singapore's first Olympic gold medal was won by Joseph Schooling


In [32]:
llm = get_llm()

print('Chain-Of-Thought:')
print('-----------------')
display(Markdown(generate_and_print_ChainOfThought(llm, fakeNews)))

print('\nReAct (DuckDuckGo Search):')
print('--------------------------')
react = generate_and_print_ReAct(llm, "ddg-search",fakeNews)
print("Input:")
display(Markdown(react['input']))
print("Output:")
display(Markdown(react['output']))

print('\nReAct (Wikipedia):')
print('--------------------------')
react = generate_and_print_ReAct(llm, "wikipedia",fakeNews)
print("Input:")
display(Markdown(react['input']))
print("Output:")
display(Markdown(react['output']))

Chain-Of-Thought:
-----------------


> Entering new SimpleSequentialChain chain...
- Joseph Schooling is a male swimmer.
- Joseph Schooling is a Singaporean citizen.
- Joseph Schooling won the gold medal in an individual event.
- Joseph Schooling won the gold medal in a swimming event.
- The Olympics are a sporting event.
- The Olympics are held every four years.
- The Olympics are a global event.
- The Olympics are open to all countries.
- The Olympics are a prestigious event.
- Winning an Olympic gold medal is a significant achievement.
- Joseph Schooling is a male swimmer. **True**
- Joseph Schooling is a Singaporean citizen. **True**
- Joseph Schooling won the gold medal in an individual event. **True**
- Joseph Schooling won the gold medal in a swimming event. **True**
- The Olympics are a sporting event. **True**
- The Olympics are held every four years. **True**
- The Olympics are a global event. **True**
- The Olympics are open to all countries. **True**
- The Olympics are a p

TRUE


ReAct (DuckDuckGo Search):
--------------------------
Inside generate_and_print: q = Singapore's first Olympic gold medal was won by Joseph Schooling


> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: Singapore's first Olympic gold medalJoseph Schooling, who beat Michael Phelps in the 100-meter butterfly to win Singapore's first and only Olympic gold medal at Rio de Janeiro in 2016, announced his retirement on Tuesday. It was the ... Singapore swimmer Joseph Schooling with his men's 100m butterfly gold medal at the 2016 Rio de Janeiro Olympics. (PHOTO: Clive Rose/Getty Images) SINGAPORE — A monumental sporting career has ended, as Joseph Schooling - Singapore's first and only Olympic gold medallist - announced on Tuesday (2 April) that he is retiring from competitive ... SINGAPORE: Joseph Schooling made history for Singapore when he won the country's first Olympic gold medal at the Rio Games in 2016. He had beaten US legend Michael Phelps in the 100m butte

Here is a statement:
    Singapore's first Olympic gold medal was won by Joseph Schooling
    Is this statement is correct? You can use tools to find information if needed.
    The final response is FALSE if the statement is FALSE. Otherwise, TRUE.

Output:


TRUE


ReAct (Wikipedia):
--------------------------
Inside generate_and_print: q = Singapore's first Olympic gold medal was won by Joseph Schooling


> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: Singapore's first Olympic gold medalPage: Singapore at the Olympics
Summary: Singapore has sent athletes to the celebration of the Olympic Games since 1948, when it was established as a separate British crown colony from the Straits Settlements just over three months before the commencement of the 1948 Summer Olympics.
It continued to send a team to the Games until 1964 when the Singaporean delegation competed with Malaysia, which sent a combined team.
Upon Singapore's full independence from Malaysia in 1965, the country continued to participate in all subsequent editions of the Summer Olympics except in 1980 when the country participated in a large United States–led boycott against the Soviet invasion of Afghanistan.
Singapore made their Winter Olympics debut at the 2018 Wi

Here is a statement:
    Singapore's first Olympic gold medal was won by Joseph Schooling
    Is this statement is correct? You can use tools to find information if needed.
    The final response is FALSE if the statement is FALSE. Otherwise, TRUE.

Output:


TRUE